PART A

In [309]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [310]:
df = pd.read_csv(r'framingham_heart_disease.csv')
df = df[['male', 'age', 'BMI', 'heartRate', 'sysBP']]
original_size = df.shape[0]
df.dropna(how='any', inplace=True)
no_null_size = df.shape[0]
df.insert(loc=0, column='intercept', value=1)
print(f'Data size after dropping Null values is'
 f' {no_null_size / original_size * 100:0.3}% of the original data size')

Data size after dropping Null values is 99.5% of the original data size


In [311]:
def generate_sample(df, size):
    np.random.seed(555)
    flag = True
    sample = None
    while flag: #will take another sample if there is one sex only
        sample = df.sample(size,ignore_index=True)
        counts_gender = sample[['male', 'sysBP']].groupby('male').count()
        male_count = counts_gender['sysBP'][1]
        female_count = counts_gender['sysBP'][0]
        if female_count != 0 and male_count != 0:
            if abs(female_count - male_count) <= size * 0.1:
                flag = False
    return sample

sample = generate_sample(df, 200)
X_variables = ['intercept','age', 'BMI', 'heartRate']
y_variable = 'sysBP'
X = df[X_variables]
y = df[y_variable]
X_sample = sample[X_variables]
y_sample = sample[y_variable]

In [312]:
import scipy.stats as stats
z_alpha = stats.norm.ppf(0.975)
#Q1.a
def calculate_beta(X, y):
    # Calculate MLE
    C = X.T @ X
    C_inv = np.linalg.inv(C)
    C_inv_X = C_inv @ X.T
    beta = C_inv_X @ y
    return np.round(beta, 4).to_numpy()

def beta_CI(X, y, beta_sample, x_variables):
    n = X.shape[0]
    p = X.shape[1]
    C = np.linalg.inv(X.T @ X)
    e = y - np.dot(X,beta_sample) #residuals
    res_var_estimate = (1 / (n - p)) * (e.T @ e) #sigma-hat squared
    CI = []
    for i, variable in enumerate(x_variables):
        std_estimate = np.sqrt(res_var_estimate * (C[i][i])) #SE of Beta_i
        CI.append(np.round([beta_sample[i] - z_alpha * std_estimate,
                            beta_sample[i] + z_alpha * std_estimate], 4))
    return CI

In [313]:
#Q1.a
beta_sample = calculate_beta(X_sample,y_sample)
regular_CI = beta_CI(X_sample, y_sample, beta_sample,X_variables)
print(f'Beta = {beta_sample}')
for i in range(beta_sample.size):
    print(f"CI for beta_{i} is: {regular_CI[i]}")


Beta = [41.7462  0.9039  0.785   0.3429]
CI for beta_0 is: [15.3282 68.1642]
CI for beta_1 is: [0.6169 1.1909]
CI for beta_2 is: [0.0966 1.4734]
CI for beta_3 is: [0.1438 0.542 ]


In [314]:
#Q1.b
B = 400
bootstrap_beta_hist = []
bootstrap_beta_var = []
np.random.seed(555)
for i in range(B):
    bootstrap_data = sample.sample(frac=1,replace=True) # sampling 200 rows with returns
    X_i = bootstrap_data[X_variables]
    y_i = bootstrap_data[y_variable]
    bootstrap_beta = calculate_beta(X_i,y_i)
    bootstrap_beta_hist.append(bootstrap_beta)
    bootstrap_beta_var.append(bootstrap_beta)
beta_se = np.std(bootstrap_beta_var,axis=0)
normal_CI = []
for i in range(X_sample.shape[1]):
    CI = np.round([beta_sample[i] - z_alpha * beta_se[i],
                        beta_sample[i] + z_alpha * beta_se[i]], 4)
    normal_CI.append(CI)
    print(f"Normal based CI for beta_{i} is: {CI}")

Normal based CI for beta_0 is: [20.7928 62.6996]
Normal based CI for beta_1 is: [0.6238 1.184 ]
Normal based CI for beta_2 is: [0.007 1.563]
Normal based CI for beta_3 is: [0.1655 0.5203]


In [315]:
#Q1.c
bootstrap_beta_hist = np.array(bootstrap_beta_hist)
pivotal_CI = []
for i,(beta_est,beta_bootstrap) in enumerate(zip(beta_sample,bootstrap_beta_hist.T)):
    beta_quantiles = np.quantile(beta_bootstrap,[0.025,0.975])
    CI = np.round([2*beta_est - beta_quantiles[1], 2*beta_est - beta_quantiles[0]],4)
    pivotal_CI.append(CI)
    print(f"Pivotal CI for beta_{i} is: {CI}")

Pivotal CI for beta_0 is: [22.9332 63.2129]
Pivotal CI for beta_1 is: [0.6028 1.1811]
Pivotal CI for beta_2 is: [0.0458 1.5722]
Pivotal CI for beta_3 is: [0.1641 0.4998]


In [316]:
#Q1.d
quantile_CI = []
for i,(beta_est,beta_bootstrap) in enumerate(zip(beta_sample,bootstrap_beta_hist.T)):
    beta_quantiles = np.quantile(beta_bootstrap,[0.025,0.975])
    CI = np.round([beta_quantiles[0], beta_quantiles[1]],4)
    quantile_CI.append(CI)
    print(f"Pivotal CI for beta_{i} is: {CI}")

Pivotal CI for beta_0 is: [20.2795 60.5592]
Pivotal CI for beta_1 is: [0.6267 1.205 ]
Pivotal CI for beta_2 is: [-0.0022  1.5242]
Pivotal CI for beta_3 is: [0.186  0.5217]


In [317]:
#Q2
regular_CI_lengths = [ci[1]-ci[0] for ci in regular_CI]
normal_CI_lengths = [ci[1]-ci[0] for ci in normal_CI]
pivotal_CI_lengths = [ci[1]-ci[0] for ci in pivotal_CI]
quantile_CI_lengths = [ci[1]-ci[0] for ci in quantile_CI]
data = list(zip(regular_CI_lengths,normal_CI_lengths,pivotal_CI_lengths,quantile_CI_lengths))
CI_lengths_comparison = pd.DataFrame(data,index=['regular','normal','pivotal','quantile'],columns=['beta 0','beta 1', 'beta 2', 'beta 3'])

In [318]:
def CI_contains(CI,type,actual_betas):
    for i,ci in enumerate(CI):
        decision = 'contains' if ci[0]<=actual_betas[i]<=ci[1] else "doesn't contain"
        print(f'the {type} CI {list(ci)} {decision} beta_{i} = {actual_betas[i]}')

In [319]:
actual_betas = calculate_beta(X,y)
print("Regular CI")
CI_contains(regular_CI,'regular',actual_betas)
print("\nNormal CI")
CI_contains(normal_CI,'normal',actual_betas)
print("\nPivotal CI")
CI_contains(pivotal_CI,'pivotal',actual_betas)
print("\nQuantile CI")
CI_contains(quantile_CI,'quantile',actual_betas)

Regular CI
the regular CI [15.3282, 68.1642] contains beta_0 = 26.1297
the regular CI [0.6169, 1.1909] contains beta_1 = 0.9203
the regular CI [0.0966, 1.4734] contains beta_2 = 1.4356
the regular CI [0.1438, 0.542] contains beta_3 = 0.3102

Normal CI
the normal CI [20.7928, 62.6996] contains beta_0 = 26.1297
the normal CI [0.6238, 1.184] contains beta_1 = 0.9203
the normal CI [0.007, 1.563] contains beta_2 = 1.4356
the normal CI [0.1655, 0.5203] contains beta_3 = 0.3102

Pivotal CI
the pivotal CI [22.9332, 63.2129] contains beta_0 = 26.1297
the pivotal CI [0.6028, 1.1811] contains beta_1 = 0.9203
the pivotal CI [0.0458, 1.5722] contains beta_2 = 1.4356
the pivotal CI [0.1641, 0.4998] contains beta_3 = 0.3102

Quantile CI
the quantile CI [20.2795, 60.5592] contains beta_0 = 26.1297
the quantile CI [0.6267, 1.205] contains beta_1 = 0.9203
the quantile CI [-0.0022, 1.5242] contains beta_2 = 1.4356
the quantile CI [0.186, 0.5217] contains beta_3 = 0.3102


In [320]:
#Q3 - new sample
df_new = df.merge(sample, how='left', indicator=True)
df_new = df_new[df_new['_merge'] == 'left_only']
new_sample = generate_sample(df_new,100)
new_X_sample = new_sample[X_variables]
new_y_sample = new_sample[y_variable]

In [321]:
#Q3.a - prediction
new_beta_sample = calculate_beta(new_X_sample,new_y_sample)
new_y_pred = np.matmul(new_X_sample, new_beta_sample)

In [322]:
#Q3.b - CI for E[y_new|x_new]
B = 400
prediction_hist = []
for i in range(B):
    bootstrap_data = new_sample.sample(frac=1,replace=True) # sampling 200 rows with returns
    X_i = bootstrap_data[X_variables]
    y_i = bootstrap_data[y_variable]
    bootstrap_beta = calculate_beta(X_i,y_i)
    bootstrap_y_pred = np.matmul(X_i,bootstrap_beta)
    prediction_hist.append(bootstrap_y_pred)

prediction_hist = np.array(prediction_hist)
pred_se = np.std(prediction_hist,axis=0)
pred_mean = np.mean(prediction_hist,axis=0)
normal_CI = []
for i in range(prediction_hist.shape[1]):
    CI = np.round([pred_mean[i] - z_alpha * pred_se[i],
                        pred_mean[i] + z_alpha * pred_se[i]], 4)
    normal_CI.append(CI)


In [335]:
#Q3.c
actual_confidence = np.mean([ int(ci[0]<=pred<=ci[1])
                              for ci,pred in zip(normal_CI,new_y_pred)])
print(f'{actual_confidence}% of the Bootstraped-CIs contain the predicted value of Ynew,\n'
      f'which is {np.abs(actual_confidence-0.95):.3} far from the desired confidence level of 95%')

0.96% of the Bootstraped-CIs contain the predicted value of Ynew,
which is 0.01 far from the desired confidence level of 95%


#Q3.d<br>
We will use the formula from Tirgul 5:<br>
$CI(Y_{new}|x_{new})=\hat{Y}_{new}\ \pm\ Z_{\frac{\alpha}{2}}\sqrt{\hat{\sigma^2_{\epsilon}} \cdot X_{new} \cdot C \cdot X_{new}^T\ + \hat{\sigma^2_{\epsilon}}}$<br>
So, we will calculate $\hat{\sigma^2_{\epsilon}}$ within every bootstrap sample(aka the noise variance) and add it to the variance of the prediction $Var(\hat{Y}_{new})$.<br>
Normal estimation is still valid (as we used it in the CI of $E[\hat{Y}_{new}|\hat{x}_{new}]$) because the "noise" is normally distributed too.